In [ ]:
import csv
import pandas as pd
import os
import statistics

def cluster_character(cluster, host_ip):
    ret = []
    flow_dir = [] 
    flow = []
    request = []
    response = []
    
    for record in cluster:
        if record[2] == host_ip: 
            flow_dir.append(int(record[5]) * 1)
            request.append(int(record[5]))
        else:
            flow_dir.append(int(record[5]) * -1)
            response.append(int(record[5]))
        flow.append(int(record[5]))
    packet_num = len(flow) 
    packet_sum = sum(flow) 
    
    request_num = len(request) 
    request_sum = sum(request)
    response_num = len(response) 
    response_sum = sum(response)

    if request:
        rate_num = response_num/request_num 
        rate_sum = response_sum/request_sum 
    else:
        rate_num = -1
        rate_sum = -1
    
    def cal_vari(packets):
        if len(packets) >= 2:
            ans = statistics.stdev(packets)/len(packets)
        else:
            ans = -1
        return ans
    
    request_vari = cal_vari(request)
    response_vari = cal_vari(response)
    all_vari = cal_vari(flow)
        
    ret.append(packet_num)
    ret.append(packet_sum)
    ret.append(packet_sum/packet_num) 
    ret.append(rate_num)
    ret.append(rate_sum) 
    ret.append(all_vari)
    ret.append(request_vari)
    if request:
        ret.append(request_sum/request_num)
    else:
        ret.append(-1)
    ret.append(response_vari)
    if response:
        ret.append(response_sum/response_num)
    else:
        ret.append(-1)
    return ret

def time_cluster_feature(file_path, time, host_ip):
    input_df = pd.read_csv(file_path)
    out_columns = ['packet_count','payload_size','payload_size_mean','resp_query_packet_ratio','resp_query_payload_ratio','payload_size_stddev','query_payload_stddev','query_payload_mean','resp_payload_stddev','resp_payload_mean']
    output_df = pd.DataFrame(columns = out_columns)
    tmp_packets = [] 
    tmp_time = 0 # start time

    for index, row in input_df.iterrows():
        if str(row['Info'])[:3] == 'App':
            if tmp_time == 0: # new seg
                tmp_time = float(row['Time'])
                tmp_packets.append(row.tolist())
                
            else:
                if (float(row['Time']) - tmp_time) <= time:
                    tmp_packets.append(row.tolist())
                else:
                    features = cluster_character(tmp_packets, host_ip)
                    df = pd.DataFrame([features], columns = out_columns)
                    output_df = output_df.append(df,ignore_index=True)
                    tmp_time = float(row[1])
                    tmp_packets.clear()
                    tmp_packets.append(row)
    return output_df